<a href="https://colab.research.google.com/github/steffanobrito/dataset-teste1/blob/master/Confidence_inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np

Using TensorFlow backend.


In [0]:
#gpu_options = tf.GPUOptions(allow_growth=True)
#session = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
import keras
from keras.applications import VGG16

vgg_conv = keras.applications.inception_v3.InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(299, 299, 3))

87916544/87910968 [==============================] - 3s 0us/step


In [4]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [5]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11056913896584271212, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4985209640382278004
 physical_device_desc: "device: XLA_CPU device"]

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_dir = '/home/steffano/teste1/treino'
validation_dir = '/home/steffano/teste1/teste'
nTrain = 1933
nVal = 415

In [0]:
datagen = ImageDataGenerator(rescale=1./255,
                            horizontal_flip=True,
                            zoom_range= 0.4,
                            vertical_flip=True
                            )

datagenteste = ImageDataGenerator(rescale=1./255)

batch_size = 10

train_features = np.zeros(shape=(nTrain, 8, 8, 2048))
train_labels = np.zeros(shape=(nTrain,2))

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
        
train_features = np.reshape(train_features, (nTrain, 8 * 8 * 2048))

Found 1933 images belonging to 2 classes.


In [0]:
validation_features = np.zeros(shape=(nVal, 8, 8, 2048))
validation_labels = np.zeros(shape=(nVal,2))

validation_generator = datagenteste.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break

validation_features = np.reshape(validation_features, (nVal, 8 * 8 * 2048))

Found 415 images belonging to 2 classes.


In [0]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(2048, activation='relu', input_dim=8 * 8 * 2048))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(2, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-3),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features,
                    train_labels,
                    epochs=60,
                    batch_size=batch_size,
                    validation_data=(validation_features,validation_labels))

Train on 1933 samples, validate on 415 samples
Epoch 1/60


ResourceExhaustedError: OOM when allocating tensor with shape[131072,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node dense_1/random_uniform/RandomUniform (defined at /home/steffano/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4139)  = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=4795361, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_1/random_uniform/shape)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'dense_1/random_uniform/RandomUniform', defined at:
  File "/home/steffano/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/steffano/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/steffano/anaconda3/lib/python3.6/site-packages/ipykernel_launche